In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from visdom import Visdom
viz = Visdom()

Setting up a new session...


In [3]:
batch_size = 16
learning_rate = 0.0002
num_epoch = 10

In [4]:
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

In [5]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
mnist_test.__getitem__(0)[0].size(), mnist_test.__len__()

torch.Size([1, 28, 28]) 60000


(torch.Size([1, 28, 28]), 10000)

In [12]:
train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size, shuffle=True,num_workers=2,drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size, shuffle=False,num_workers=2,drop_last=True)

In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer = nn.Sequential(
        nn.Conv2d(1,16,5), nn.ReLU(),
        nn.Conv2d(16,32,5), nn.ReLU(), nn.MaxPool2d(2,2),
        nn.Conv2d(32,64,5), nn.ReLU(), nn.MaxPool2d(2,2))
        self.fc_layer = nn.Sequential(
        nn.Linear(64*3*3, 100),
        nn.ReLU(),
        nn.Linear(100,10))
    
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        
        return out
    
model = CNN().cuda()

In [10]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [13]:
for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        x = Variable(image).cuda()
        y_ = Variable(label).cuda()
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
        if j % 1000 == 0:
            print(loss)

tensor(2.2935, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2969, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2977, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3002, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2934, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2826, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2948, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2858, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2626, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2453, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2635, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2307, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2130, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2245, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.1561, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.0503, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.1314, device='cuda:0', grad_fn=<NllLossBackward

In [15]:
correct = 0
total = 0

for image, label in test_loader:
    x = Variable(image, volatile=True).cuda()
    y_ = Variable(label).cuda()
    
    output = model.forward(x)
    _, output_index = torch.max(output, 1)
    
    total += label.size(0)
    correct += (output_index == y_).sum().float()
    
print("Accuracy of Test Data : {}".format(100*correct/total))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """


Accuracy of Test Data : 91.06999969482422
